In [16]:
from mr_eval.utils.utils import *
from typing import List, Dict, Any
sft_data = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/reasoning/tool_dataset/datasets/chargemma_final/select/chartgemma-reachqa-combined-sharegpt.json"
sft_data = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/reasoning/tool_dataset/datasets/chargemma_final/format_validation/chartgemma-reachqa-combined-sharegpt-llama3.1correct.json"
sft_data = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/reasoning/tool_dataset/datasets/chargemma_final/format_validation/opensource_version/chartgemma-combined-sharegpt.json"
sft_data = "/mnt/petrelfs/songmingyang/songmingyang/data/mm/reasoning/tool_dataset/datasets/chargemma_final/format_validation/opensource_version/chartgemma-combined-sharegpt-llama3.1correct.json"
sft_data = load_json_file(sft_data)
len(sft_data)


2942

In [12]:
def format_reward(predict_str: List, ground_truth: str):
    """Reward function that checks if the completion has a specific format, in this case, if it contains the word 'Terminate'."""
    reward = 0.0
    answer = None
    sub_format_rewards = []
    for pstr in predict_str:
        try:
            current_reward = 0.0
            temp_dict = json.loads(pstr)
            assert "thought" in temp_dict and "actions" in temp_dict
            
            current_reward = 1.0
        except:
            current_reward = 0.0
        sub_format_rewards.append(current_reward)
        
    reward = sum(sub_format_rewards) / len(sub_format_rewards) if sub_format_rewards else 0.0
    
    last_pstr = predict_str[-1]
    try:
        temp_dict = json.loads(last_pstr)
        assert "thought" in temp_dict and "action" in temp_dict
        action = temp_dict["action"][-1]
        assert action["name"] == "Terminate"
        answer = action["arguments"]["ans"]
    except:
        answer = None

    return answer, reward

In [13]:
sft_data[0]["conversations"][1]["value"]
sft_data[0]

{'conversations': [{'from': 'human',
   'value': 'You are a visual assistant capable of generating and solving steps for chart-based reasoning. Your goal is to answer chart-related questions. You can rely on your own capabilities or use external tools to assist in solving. The available actions include: OCR, Point, DrawHorizontalLineByY, DrawVerticalLineByX, ZoomInSubfigure, SegmentRegionAroundPoint, and Terminate.\nYour output should be in a strict JSON format as follows:\n{"thought": "the reasoning process", "actions": [{"name": "action", "arguments": {"argument1": "value1", "argument2": "value2"}}]}\n\n<image>\n\nQuestion: Look at Female Householders with related children under 18, Find the red bar, its value 20%, What is that bar  ?'},
  {'from': 'gpt',
   'value': '{"thought": "To identify the label of the red bar representing 20% in the \\"Female Householders with related children under 18\\" category, I need to reference the legend in the chart that correlates colors with educat

In [14]:
format_acc_values = []

for item in sft_data:
    conversations = item["conversations"]
    model_outputs = []
    for conv in conversations:
        if conv["from"] == "gpt":
            model_response = conv["value"]
            model_outputs.append(model_response)
    format_acc = format_reward(model_outputs, None)[1]
    if format_acc == 1.0:
        
        format_acc_values.append(format_acc)
    else:
        format_acc_values.append(0)

max_format_acc = max(format_acc_values)
min_format_acc = min(format_acc_values)
mean_format_acc = sum(format_acc_values) / len(format_acc_values)
print(f"Average format accuracy: {mean_format_acc:.4f}")


Average format accuracy: 0.9905


In [15]:
import json
format_acc_values = []

for item in sft_data:
    conversations = item["conversations"]
    for conv in conversations:
        if conv["from"] == "gpt":
            model_response = conv["value"]
            try:
                temp_dict = json.loads(model_response)
                thoughts = temp_dict["thought"]
                actions = temp_dict["actions"]
                format_acc_values.append(1)
            except:
                format_acc_values.append(0)

max_format_acc = max(format_acc_values)
min_format_acc = min(format_acc_values)
mean_format_acc = sum(format_acc_values) / len(format_acc_values)
print(f"Average format accuracy: {mean_format_acc:.4f}")

Average format accuracy: 0.9963
